# Exploration of text-to-speech methods


### Transcript generation
Generating a text transcript for the podcast episode based on the provided URL:
- assigning timestamps to uterances (when it was said)
- assigning speakers to uterances (who said what)

Install the needed libraries

In [ ]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers torch torchvision torchaudio pytube pydub pyannote.audio peft
!pip uninstall speechbox && pip install git+https://github.com/huggingface/speechbox.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.4 MB/s 

Download the podcast from youtube and convert it to .wav file

In [ ]:
# Download the audio from YouTube and save to a file
import os
from pytube import YouTube
youtube_url = "https://www.youtube.com/watch?v=_yZiNnQftxU&ab_channel=TheDiaryOfACEO"
yt = YouTube(youtube_url)
audio_stream = yt.streams.filter(only_audio=True).first()
audio_file = audio_stream.download()

# Convert to .wav
base, ext = os.path.splitext(audio_file)
wav_audio = "audio" + '.wav'
os.rename(audio_file, wav_audio)

Performing combined speech recognition and diarization, using ASRDiarizationPipeline from the speechbox library with `openai/whisper-large` for speech recognition and `pyannote/speaker-diarization-3.1 `for the speaker diarizaition. I tested several different models, but this was the best, generating almost a perfect transcript and good diarization, while also combining the result.

In [ ]:
import torch
from speechbox import ASRDiarizationPipeline

# Load the model on CUDA cores
device = "cuda:0" if torch.cuda.is_available() else "cpu"

#Create the pipeline for speech recognition and speaker diarization
pipeline = ASRDiarizationPipeline.from_pretrained("openai/whisper-large", device=device, use_auth_token=os.environ['HF_AUTH'])

out = pipeline(wav_audio)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
# Create dataframe based on the model output
import pandas as pd
df_transcript = pd.DataFrame.from_dict(out)
df_transcript = df_transcript[['timestamp','speaker', 'text']]

In [ ]:
df_transcript

,timestamp,speaker,text
0,"(0.0, 7.5)",SPEAKER_01,We just spent half hour talking about this an...
1,"(7.5, 13.4)",SPEAKER_00,World renowned astrophysicist turned TV host....
2,"(13.4, 35.0)",SPEAKER_01,"And of course, best-selling author. In the la..."
3,"(35.0, 39.0)",SPEAKER_00,What are the things that you are most concern...
4,"(39.0, 53.04)",SPEAKER_01,"A lot of things. So, there's some delusional ..."
...,...,...,...
91,"(6075.76, 6077.48)",SPEAKER_00,"and then there's the first human,"
92,"(6077.48, 6357.52)",SPEAKER_01,"and then there's the first human, and then th..."
93,"(6357.52, 6372.76)",SPEAKER_00,Thank you for the generosity of your time. Th...
94,"(6372.76, 6483.48)",SPEAKER_01,And can I end with the beginning quote of the...


In [ ]:
# Save dataframe to csv
df_transcript.to_csv("1A.csv", index=False)

### Chapter division
Developing a model that divides the text transcript in smaller segments based on the topic being discussed — topic detection.

Loading a language model and prompting it to assign a topic name to each section of the transcript. This is tricky as I did not have enough resources to just prompt the model to divide the whole transcript (I was getting the CUDA out of memory error and the smaller models did not have a context window big enough). Therefore I splitted the transcript in chunks and made the model assign a topic to each chunk. I used the `meta-llama/Llama-2-7b-chat-hf` LLM.




In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# Download the model from the HuggingFace hub
hf_auth = os.environ['HF_AUTH']
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1067: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [ ]:
# Load the tokenizer for the model
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Create the pipeline for text generation
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.0001,  # very low temperature to avoid hallucination
    max_new_tokens=10,  # low number of tokens as it just assigns the topic name
    repetition_penalty=1.5  # without this output begins repeating
)

In [ ]:
# Create a prompt template. I used a FewShot approach to show the model what I want to achieve by prompting a few examples before the actual query.
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
prompt_topic = PromptTemplate(input_variables=["transcript"],
                                template=
"""
You will be provided with a few examples of assigning topic to a transcript. You will have to do the same on a given example. Provide just the topic of the transcript, with maximum 4 words.

transcript: "With iPhone 15 Pro, you have multiple focal lengths to work with. It's like having seven pro lenses in your pocket, everywhere you go. The 48MP Main camera is more advanced than ever, capturing super‑high‑resolution photos with a new level of detail and color. You'll see the improvements in your portraits.?"
section_name: iPhone 15 Pro Camera

transcript: A Japanese spacecraft just took a huge step toward pulling off the nation's first-ever moon landing. Japan's robotic SLIM moon lander arrived in lunar orbit on Christmas Day (Dec. 25) as planned, the Japan Aerospace Exploration Agency announced. The spacecraft entered lunar orbit at 2:51 a.m. EDT (4:51 p.m. Japan Standard Time, 0751 GMT).
section_name: Japanese Spacecraft

transcript: {transcript}
section_name: """)

In [ ]:
# Initialize the model with a given pipeline
llm = HuggingFacePipeline(pipeline=generate_text)
sections = LLMChain(llm=llm, prompt=prompt_topic, output_key="section_name")

In [ ]:
# Assign topics to a list
topics = []

for i in range(1,20):
  transcript = [' '.join(df_transcript['text'][(i-1)*5:i*5])]
  output = sections(transcript[0])
  print(output["section_name"])
  topics.append(output["section_name"])

 Living Forever
 Origin Story
 Non Bitter Influences
 Resiliance
 Race
 Race Relations
 Racism
 Attack Culture
 Educating Through Conflict
🚀Cosmic Ramblings


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Kinship With  


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Transit Of Mercury


...

 Stars


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Perspective On Death


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Translating Topics


In [ ]:
# Create a new dataframe with the topics assigned
import ast
df_topics = pd.DataFrame(columns=['timestamp', 'text', 'topic'])
for i in range(1,20):
  transcript = [' '.join(df_transcript['text'][(i-1)*5:i*5])]
  timestamp_start = df_transcript["timestamp"][(i-1)*5]
  timestamp_start = ast.literal_eval(timestamp_start)
  timestamp_start = timestamp_start[0]

  timestamp_end = df_transcript["timestamp"][i*5]
  timestamp_end = ast.literal_eval(timestamp_end)
  timestamp_end = timestamp_end[1]

  timestamp = (timestamp_start, timestamp_end)
  df_topics = df_topics.append({"timestamp": timestamp, "text": transcript[0], "topic": topics[i-1]}, ignore_index = True)


In [ ]:
df_topics.to_csv("1B.csv", index=False)
df_topics

,timestamp,text,topic
0,"(0.0, 54.72)",We just spent half hour talking about this an...,Living Forever
1,"(53.04, 346.92)",Did you see what happened with Neil deGrasse ...,Origin Story
2,"(142.58, 508.0)","Well, this is very Marvel comics of you. What...",Non Bitter Influences
3,"(503.46, 702.16)",Going forward. I saw the emotion in your face...,Resiliance
4,"(696.88, 775.2)",Do they want to have a conversation to explor...,Race
5,"(764.64, 1019.8)","in the 1960s, 70s from my guests. And it's he...",Race Relations
6,"(1001.36, 1448.22)","Yeah, people didn't, yeah, and again, this is...",Racism
7,"(1415.48, 1851.16)",So let's talk about a separate point. What ad...,Attack Culture
8,"(1815.76, 2242.9)",It's never my intent to post a tweet that get...,Educating Through Conflict
9,"(2225.1, 2574.08)",Why don't people do that? Why aren't they? I ...,🚀Cosmic Ramblings


### Summary of chapters

I used the same apporach as previously. I just prompted it to create a summary for each chunk of the transcript. I add stopping criteria for the model so it does not ramble and stops when it reaches a '.' so it does not generate too long summaries

In [ ]:
# List stopping characters and words
stop_list = ['\nHuman:', '\n```\n', '.', ';']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]

In [ ]:
import torch
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
# Create the pipeline
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.1,  # Slightly higher temperature for bigger creativity
    max_new_tokens=40,  # More tokens needed for a summary
    repetition_penalty=1.5  # without this output begins repeating
)

In [ ]:
# Changing the prompt for the use-case
prompt_summary = PromptTemplate(input_variables=["transcript"],
                                template=
"""
You will be provided with a few examples of writing a summary of a transcript. You will have to do the same on a given example. Provide a short summary of the text given in the transcript. The summary should not be longer than one sentence.

transcript: With iPhone 15 Pro, you have multiple focal lengths to work with. It's like having seven pro lenses in your pocket, everywhere you go. The 48MP Main camera is more advanced than ever, capturing super‑high‑resolution photos with a new level of detail and color. You'll see the improvements in your portraits.?
summary: Description of a new IPhone 15 Pro that provides information about the technical details of the cameras.

transcript: A Japanese spacecraft just took a huge step toward pulling off the nation's first-ever moon landing. Japan's robotic SLIM moon lander arrived in lunar orbit on Christmas Day (Dec. 25) as planned, the Japan Aerospace Exploration Agency announced. The spacecraft entered lunar orbit at 2:51 a.m. EDT (4:51 p.m. Japan Standard Time, 0751 GMT).
summary: Description of a first-ever moon landing performed by Japan.

transcript: {transcript}
summary: """)

In [ ]:
llm = HuggingFacePipeline(pipeline=generate_text)
summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

In [ ]:
# Create the dataframe
df_summaries = df_topics.copy()
summaries = []
for i in range(0,19):
  transcript = df_summaries["text"][i]
  output = summary(transcript)
  print(output["summary"])
  summaries.append(output["summary"])

df_summaries["summaries"] = summaries

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 An interview between two individuals discussing topics such as immortality through advancements in technology; how society might change based upon achieving "escape velocity" or living past traditional lifespans


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 An interview between two individuals discussing science topics such astrophysics and psychology; they also talk bout how being biased affect scientific findings


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

 An interview discussing how frequently someone cries due to various factors including empathy towards others, witnesses actsofkindnessthat evoke feelingsofhopefortheworldandimm


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 An interview discusses how Galileo faced opposition for his theory challenged traditional religious views regarding geocentrism; now, modern scientists face similar conflicts due to their research findings contradictory


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Transcription discusses various topics such as longevity, living conditions, animal behavior, personal growth,and intergalactic travel.The speaker talks about ways animals perceive their surroundings


In [ ]:
df_summaries.to_csv('1C.csv', index=False)
df_summaries

,timestamp,text,topic,summaries
0,"(0.0, 54.72)",We just spent half hour talking about this an...,Living Forever,An interview between two individuals discussi...
1,"(53.04, 346.92)",Did you see what happened with Neil deGrasse ...,Origin Story,An interview between two individuals discussi...
2,"(142.58, 508.0)","Well, this is very Marvel comics of you. What...",Non Bitter Influences,An individual reflects on their childhood exp...
3,"(503.46, 702.16)",Going forward. I saw the emotion in your face...,Resiliance,Discussion between two individuals regarding ...
4,"(696.88, 775.2)",Do they want to have a conversation to explor...,Race,An excerpt where someone talks against racial...
5,"(764.64, 1019.8)","in the 1960s, 70s from my guests. And it's he...",Race Relations,An interview between Barack Obama & Ta Nehisi...
6,"(1001.36, 1448.22)","Yeah, people didn't, yeah, and again, this is...",Racism,Dr. Neill De Grass Tyson shares his personal ...
7,"(1415.48, 1851.16)",So let's talk about a separate point. What ad...,Attack Culture,Dr. Neil deGrasse Tyson discusses his concern...
8,"(1815.76, 2242.9)",It's never my intent to post a tweet that get...,Educating Through Conflict,Elon Musk discusses his views regarding curre...
9,"(2225.1, 2574.08)",Why don't people do that? Why aren't they? I ...,??Cosmic Ramblings,Discusses public opinion regarding safety con...


### Face tracking

In [ ]:
!pip install pytube ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 934.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 9.2 MB/s eta 0:00:00


In [ ]:
# Download the video from YouTube and save to a file
import os
from pytube import YouTube
youtube_url = "https://www.youtube.com/watch?v=_yZiNnQftxU&ab_channel=TheDiaryOfACEO"
yt = YouTube(youtube_url)
stream = yt.streams.filter(progressive = True,
file_extension = "mp4").first()
file = stream.download()

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
cap = cv2.VideoCapture(file)

# Store the track history
track_history = defaultdict(lambda: [])

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "face_tracking.mp4"
output_video = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, show=False, classes=0)

        if results[0].boxes.id != None:
          boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
          track_ids = results[0].boxes.id.cpu().numpy().astype(int)
          confidences = results[0].boxes.conf.cpu().numpy().astype(int)

        # # Get the boxes and track IDs
        # boxes = results[0].boxes.xywh.cpu()
        # track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Write the frame to the output video
        output_video.write(frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
0: 384x640 (no detections), 13.7ms
Speed: 1.1ms preprocess, 13.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.0ms
Speed: 1.1ms preprocess, 8.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.3ms
Speed: 1.1ms preprocess, 8.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10.2ms
Speed: 1.2ms preprocess, 10.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.8ms
Speed: 1.3ms preprocess, 9.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.6ms
Speed: 1.1ms preprocess, 9.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8.4ms
Speed: 1.2ms preprocess, 8.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.3ms
Speed: 1.1ms pr

### Removing silent moments

In [ ]:
!pip install pyannote.audio

In [ ]:
from pyannote.audio import Pipeline

diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=os.environ['HF_AUTH'])

# apply pretrained pipeline
diarization = diarization_pipeline("audio.wav")


result_speaker = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    result_speaker.append({"timestamp":(turn.start, turn.end), "speaker": speaker})

df_speaker = pd.DataFrame.from_dict(result_speaker)
df_speaker.to_csv("2B.csv", index=False)
df_speaker


/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]